In [2]:
print(os.getcwd()+'\my_model.h5')

C:\Users\Vassilis\person-blocker\my_model.h5


In [8]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras.models import load_model
from keras import backend as K
import os
import os.path as osp
from classes import InferenceConfig
import model as modellib
import tensorflow as tf

# model = load_model(os.getcwd()+'\my_model.h5')
# Required to load model, but otherwise unused
ROOT_DIR = os.getcwd()
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

MODEL_DIR = os.path.join(ROOT_DIR, "logs")  # Required to load model

if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Load model and config
config = InferenceConfig()
saved_model = modellib.MaskRCNN(mode="inference",
                          model_dir=MODEL_DIR, config=config)
saved_model.load_weights(COCO_MODEL_PATH, by_name=True)

# Get keras model and save
model = saved_model.keras_model

nb_classes = 1 # The number of output nodes in the model
prefix_output_node_names_of_final_network = 'output_node'

K.set_learning_phase(0)

pred = [None]*nb_classes
pred_node_names = [None]*nb_classes
for i in range(nb_classes):
    pred_node_names[i] = prefix_output_node_names_of_final_network+str(i)
    pred[i] = tf.identity(model.output[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)

sess = K.get_session()
output_fld = 'tensorflow_model/'
if not os.path.isdir(output_fld):
    os.mkdir(output_fld)
output_graph_name = os.getcwd() + '.pb'
output_graph_suffix = '_inference'

constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
graph_io.write_graph(constant_graph, output_fld, output_graph_name, as_text=False)
print('saved the constant graph (ready for inference) at: ', osp.join(output_fld, output_graph_name))


output nodes names are:  ['output_node0']
INFO:tensorflow:Froze 662 variables.
INFO:tensorflow:Converted 662 variables to const ops.
saved the constant graph (ready for inference) at:  C:\Users\Vassilis\person-blocker.pb
